In [5]:
#%tensorflow_version 2.0
import tensorflow as tf
tf.test.is_gpu_available()


UsageError: Cell magic `%%tf_version` not found.


Initializing parameters:

In [2]:
weight_decay = 1e-4
batch_size = 32
input_size = 160 #160x160 matrix
num_classes = 2  # binary classifier
num_features = 32 #may be much more
num_epochs = 20 

In [3]:
def vgg_block(x,filters,layers):
  for i in range(layers):
    # 2-D Convolution (with l1 & l2 Regularization: https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c)
    x = tf.keras.layers.Conv2D(filters,kernel_size = (3,3), padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer = tf.keras.regularizers.l2(weight_decay))(x)
    # Batch Normalization: https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c
    x = tf.keras.layers.BatchNormalization()(x)
    # Activavtion Function (ReLU (Linear)): https://www.tinymind.com/learn/terms/relu
    x = tf.keras.layers.Activation('relu')(x)
  return x




In [4]:
def vgg8():
  input = tf.keras.layers.Input(shape = (input_size, input_size, 3)) # channels = 3 (RGB image)
  x = vgg_block(input, 16, 2)
  x = tf.keras.layers.MaxPool2D(pool_size = (2,2))(x) # 2-D Max Pooling: https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/blocks/2d-max-pooling-block
  x = vgg_block(x, 32, 2) 
  x = tf.keras.layers.MaxPool2D(pool_size = (2,2))(x)
  x = vgg_block(x, 64, 2) 
  x = tf.keras.layers.MaxPool2D(pool_size = (2,2))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(0.4)(x) # Dropout Function: https://towardsdatascience.com/machine-learning-part-20-dropout-keras-layers-explained-8c9f6dc4c9ab
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(num_features, kernel_initializer = 'he_normal', kernel_regularizer = tf.keras.regularizers.l2(weight_decay))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  output = tf.keras.layers.Dense(num_classes, activation = 'softmax', kernel_regularizer = tf.keras.regularizers.l2(weight_decay))(x)
  model = tf.keras.models.Model(input,output)
  model.summary()
  return model


In [5]:
model = vgg8() 


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 160, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 160, 160, 16)      64        
_________________________________________________________________
activation (Activation)      (None, 160, 160, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 160, 160, 16)      2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 160, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 160, 160, 16)      0     

In [6]:
url = 'https://bit.ly/33n6FcG'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin = url, extract = True)
path_to_zip

'/home/batu/.keras/datasets/cats_and_dogs_filtered.zip'

In [7]:
import os

In [8]:
dataset_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
dataset_dir

'/home/batu/.keras/datasets/cats_and_dogs_filtered'

In [9]:
!ls $dataset_dir

train  validation  vectorize.py


In [10]:
train_dir = os.path.join(dataset_dir, 'train')
train_dir

'/home/batu/.keras/datasets/cats_and_dogs_filtered/train'

In [11]:
validation_dir = os.path.join(dataset_dir, 'validation')
validation_dir

'/home/batu/.keras/datasets/cats_and_dogs_filtered/validation'

In [12]:
train_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0/255.0,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.2,
    horizontal_flip = True,
    zoom_range = 0.2
)

In [13]:
train_gen = train_aug.flow_from_directory(train_dir, target_size = (input_size, input_size), class_mode = 'categorical', shuffle = True)

Found 2000 images belonging to 2 classes.


In [14]:
validation_aug = tf.keras.preprocessing.image.ImageDataGenerator( rescale = 1.0/255.0)

In [15]:
validation_gen = validation_aug.flow_from_directory(validation_dir, target_size = (input_size,input_size), class_mode = 'categorical')

Found 1000 images belonging to 2 classes.


In [16]:
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
model.fit_generator(train_gen, epochs = num_epochs, steps_per_epoch = 2000 // batch_size, validation_data = validation_gen, validation_steps = 1000 // batch_size)

Epoch 1/20
62/62 [==============================] - 170s 3s/step - loss: 0.8213 - accuracy: 0.6123 - val_loss: 0.9037 - val_accuracy: 0.5000
Epoch 2/20
62/62 [==============================] - 159s 3s/step - loss: 0.6943 - accuracy: 0.6479 - val_loss: 0.9746 - val_accuracy: 0.5030
Epoch 3/20
62/62 [==============================] - 163s 3s/step - loss: 0.6823 - accuracy: 0.6580 - val_loss: 0.7466 - val_accuracy: 0.5605
Epoch 4/20
62/62 [==============================] - 161s 3s/step - loss: 0.6401 - accuracy: 0.6759 - val_loss: 0.6860 - val_accuracy: 0.6310
Epoch 5/20
62/62 [==============================] - 155s 2s/step - loss: 0.6289 - accuracy: 0.6931 - val_loss: 0.6264 - val_accuracy: 0.6946
Epoch 6/20
62/62 [==============================] - 160s 3s/step - loss: 0.6326 - accuracy: 0.6890 - val_loss: 0.6183 - val_accuracy: 0.7026
Epoch 7/20
62/62 [==============================] - 174s 3s/step - loss: 0.6194 - accuracy: 0.7077 - val_loss: 0.5950 - val_accuracy: 0.7268
Epoch 8/20
62